In [16]:
import json

from kafka import KafkaConsumer

In [17]:
import os
import json
import time
from collections import namedtuple
import heapq
import uuid
import pandas as pd
import s3fs
import pyarrow.parquet as pq

endpoint_url='https://storage.budsc.midwest-datascience.com'
s3 = s3fs.S3FileSystem(
    anon=True,
    client_kwargs={
        'endpoint_url': endpoint_url
    }
)

acceleration_columns = [
    'offset',
    'id',
    'ride_id',
    'uuid',
    'x',
    'y',
    'z',
#     't'
]
Acceleration = namedtuple('Acceleration', acceleration_columns)
def read_accelerations():
    df = pq.ParquetDataset(
        's3://data/processed/bdd/accelerations',
        filesystem=s3
    ).read_pandas().to_pandas()
    
    df = df[acceleration_columns].sort_values(by=['offset'])
    
    records = [Acceleration(*record) for record in df.to_records(index=False)]
    
    return records
accelerations = read_accelerations()

location_columns = [
    'offset',
    'id',
    'ride_id',
    'uuid',
    'course',
    'latitude',
    'longitude',
    'geohash',
    'speed',
    'accuracy',
#     't'
]
Location = namedtuple('Location', location_columns)
def read_locations():
    df = pq.ParquetDataset(
        's3://data/processed/bdd/locations',
        filesystem=s3
    ).read_pandas().to_pandas()
    
    df = df[location_columns].sort_values(by=['offset'])
    
    records = [Location(*record) for record in df.to_records(index=False)]
    
    return records
    
locations = read_locations()

### Configuration Parameters 

> **TODO:** Change the configuration prameters to the appropriate values for your setup.

In [18]:
config = dict(
    bootstrap_servers=['kafka.kafka.svc.cluster.local:9092'],
    first_name='Rachel',
    last_name='Nelson'
)

config['client_id'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)
config['topic_prefix'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)

config

{'bootstrap_servers': ['kafka.kafka.svc.cluster.local:9092'],
 'first_name': 'Rachel',
 'last_name': 'Nelson',
 'client_id': 'NelsonRachel',
 'topic_prefix': 'NelsonRachel'}

Create a consumer without subscribing to any particular topic

In [19]:
general_consumer = KafkaConsumer(
    bootstrap_servers=config['bootstrap_servers']
)

List all topics you are currently allowed to view

In [20]:
general_consumer.topics()

{'ChougulePushkar-accelerations',
 'ChougulePushkar-locations',
 'DoeJohn-accelerations',
 'DoeJohn-locations',
 'DoeJohn-simple',
 'GonzalezKimberly-accelerations',
 'GonzalezKimberly-joined',
 'GonzalezKimberly-locations',
 'GonzalezKimberly-simple',
 'GonzalezKimberly-windowed',
 'HoffmannLaura-accelerations',
 'HoffmannLaura-locations',
 'KaakatyRoni-accelerations',
 'KaakatyRoni-locations',
 'KalaikkovanVasanthakumar-accelerations',
 'KalaikkovanVasanthakumar-locations',
 'KanaparthiVenkata-accelerations',
 'KanaparthiVenkata-locations',
 'KohnAaron-accelerations',
 'KohnAaron-locations',
 'KolekarShilpa-accelerations',
 'KolekarShilpa-locations',
 'Last_Name_testFirst_Name_test-locations',
 'MandalAbhijit-accelerations',
 'MandalAbhijit-locations',
 'MohammedShabuddinRamizuddin-accelerations',
 'MohammedShabuddinRamizuddin-locations',
 'MuleyTushar-accelerations',
 'MuleyTushar-accelrations',
 'MuleyTushar-locations',
 'MuthusamyGaneshkumar-accelerations',
 'MuthusamyGaneshkumar-

Close the consumer, waiting indefinitely for any needed cleanup.

In [21]:
general_consumer.close()

In [22]:
def create_kafka_consumer(topics, config=config):
    bootstrap_servers = config['bootstrap_servers']
    client_id = config['client_id']
    topic_prefix = config['topic_prefix']    
    topic_list = ['{}-{}'.format(topic_prefix, topic) for topic in topics]
    
    return KafkaConsumer(
        *topic_list,
        client_id=client_id,
        bootstrap_servers=bootstrap_servers,
        auto_offset_reset='earliest',
        enable_auto_commit=False,
        value_deserializer=lambda x: json.loads(x)
    )

consumer = create_kafka_consumer(['locations', 'accelerations'])

Gets a list of this consumer's current subscriptions

In [23]:
consumer.subscription()

{'NelsonRachel-accelerations', 'NelsonRachel-locations'}

The following function prints messages from the current consumer subscriptions. It will continue until manually stopped. 

In [ ]:
def print_messages(consumer=consumer):
    try:
        for message in consumer:
                msg_metadata = 'Message metadata: {}:{}:{}'.format(
                    message.topic, message.partition, message.offset
                )

                if message.key is not None:
                    msg_key = message.key.decode('utf-8')
                else:
                    msg_key = ''
                msg_value = json.dumps(message.value, indent=2)
                msg_value = '\n'.join(['  {}'.format(value) for value in msg_value.split('\n')])

                print('Message metadata:')
                print('  Topic: {}'.format(message.topic))
                print('  Partition: {}'.format(message.partition))
                print('  Offset: {}'.format(message.offset))
                print('Message Key: {}'.format(msg_key))
                print('Message Value:')
                print(msg_value)
                print()
    except KeyboardInterrupt:
        print("STOPPING MESSAGE CONSUMER")
        
print_messages()

Message metadata:
  Topic: NelsonRachel-locations
  Partition: 0
  Offset: 0
Message Key: 8d4d1848c6124b078ae7d0798f65e12a
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

Message metadata:
  Topic: NelsonRachel-locations
  Partition: 0
  Offset: 1
Message Key: 7857e36d21134f5fbd834c5c12c73adf
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

Message metadata:
  Topic: NelsonRachel-locations
  Partition: 0
  Offset: 2
Message Key: 171193d2ae9e4eb899bf5d89181238f1
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

Message metadata:
  Topic: NelsonRachel-locations
  Partition: 0
  Offset: 3
Message Key: c4f170df0be0444aa8e0a3eae52e3688
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

Message metadata:
  Topic: NelsonRachel-locations
  Partition: 0
  Offset: 4
Message Key: 4cef1228d1464f439dfab20c868512b4
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

Message metadata:
  Topic: NelsonRachel-locations


Message metadata:
  Topic: NelsonRachel-locations
  Partition: 0
  Offset: 7
Message Key: bc213e0676904d1193b4bedd09cc6380
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

Message metadata:
  Topic: NelsonRachel-locations
  Partition: 0
  Offset: 8
Message Key: 7861cdfaff264b6480782ab650f28ea0
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

Message metadata:
  Topic: NelsonRachel-locations
  Partition: 0
  Offset: 9
Message Key: 3c443ab1e5e64d0dbaab665175fdb217
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

Message metadata:
  Topic: NelsonRachel-locations
  Partition: 0
  Offset: 10
Message Key: 745da0de8f1544b18555d8a37b4b4f46
Message Value:
  "[[1.0779125295566454, \"85c61911b7fe2ced1000c33c9e932706\", \"6760ffa3f41908695d1405b776c3e8d5\", \"dad7eae44e784b549c8c5a3aa051a8c7\", 158.203125, 40.677641336844, -73.81793000742218, \"dr5x2jpkmtcy\", 2.119999885559082, 10.0], [1.525060886522843, \"58682c5d48cad9d9e103431d773615bf\", \

Message metadata:
  Topic: NelsonRachel-locations
  Partition: 0
  Offset: 11
Message Key: 71525c3d6cd64713b4a2078f5d9ff88e
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

Message metadata:
  Topic: NelsonRachel-locations
  Partition: 0
  Offset: 12
Message Key: 9e7064f3e9f14f488efa2b84dc33b822
Message Value:
  "[[1.0779125295566454, \"85c61911b7fe2ced1000c33c9e932706\", \"6760ffa3f41908695d1405b776c3e8d5\", \"dad7eae44e784b549c8c5a3aa051a8c7\", 158.203125, 40.677641336844, -73.81793000742218, \"dr5x2jpkmtcy\", 2.119999885559082, 10.0], [1.525060886522843, \"58682c5d48cad9d9e103431d773615bf\", \"c9a2b46c9aa515b632eddc45c4868482\", \"19b9aa10588646b3bf22c9b4865a7995\", 299.619140625, 40.76287002542555, -73.96194855681718, \"dr5ruuwscttz\", 0.0, 10.0], [4.5250608865228426, \"58682c5d48cad9d9e103431d773615bf\", \"c9a2b46c9aa515b632eddc45c4868482\", \"19b9aa10588646b3bf22c9b4865a7995\", 299.619140625, 40.76287038067684, -73.96194937863832, \"dr5ruuwsctv3\", 0.0, 10.0], [

Close the consumer, waiting indefinitely for any needed cleanup.

In [ ]:
consumer.close()